<a href="https://colab.research.google.com/github/GarimaChopra/Generative_AI/blob/main/hf_transformers_ch1_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Chapter 1 : Hello Transformer**

Below example is for Sentimental Analysis. It takes an madeup customer feedback and tries to analyse it as Positive or Negative feedback.In this example we are going to deploy a language model to Model Zoo with HuggingFace Transformers and use it to generate an original passage of text in response to the customer feedback


In [45]:
# Allows us to work with Hugging Face Transformers
!pip install pipwin
!pipwin install fiona
!pip install geopandas
!pip install modelzoo-client[transformers]


/usr/local/lib/python3.10/dist-packages/pipwin/command.py:66: UserWarning: Found a non Windows system. Package installation might not work.
  warn("Found a non Windows system. Package installation might not work.")
Package `fiona` found in cache
https://download.lfd.uci.edu/pythonlibs/archived/Fiona-1.8.21-cp310-cp310-win_amd64.whl
Fiona-1.8.21-cp310-cp310-win_amd64.whl
File /root/pipwin/Fiona-1.8.21-cp310-cp310-win_amd64.whl already exists
ERROR: Fiona-1.8.21-cp310-cp310-win_amd64.whl is not a supported wheel on this platform.
Traceback (most recent call last):
  File "/usr/local/bin/pipwin", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/pipwin/command.py", line 103, in main
    cache.install(package)
  File "/usr/local/lib/python3.10/dist-packages/pipwin/pipwin.py", line 301, in install
    subprocess.check_call([executable, "-m", "pip", "install", wheel_file])
  File "/usr/lib/python3.10/subprocess.py", line 369, in check_call
    raise Cal

In [46]:
# If need to perform translation

!pip install datasets transformers[sentencepiece]


Instantiate a pipeline by calling the pipeline() function and providing the name of the task we are interested in which is Text-Classification. By default, the text-classification pipeline uses a model that’s designed for sentiment analysis, but it also supports multiclass and multilabel classification.

In [47]:
# Hugging Face transformers

from transformers import pipeline


In [48]:
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

In [49]:
import pandas as pd


Below is the input text complain

In [50]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure
from your online store in Germany. Unfortunately, when I opened the package,
I discovered to my horror that I had been sent an action figure of Megatron
instead! As a lifelong enemy of the Decepticons, I hope you can understand my
dilemma. To resolve the issue, I demand an exchange of Megatron for the
Optimus Prime figure I ordered. Enclosed are copies of my records concerning
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

**SENTIMENT ANALYSIS**

> Checks on the feedback, if it is positive or negative. This task is called sentiment analysis



In [51]:

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


The sentiment is negative which is justified as it is  a complain.

**NER TAGGING**

> Now using Name Entity Recognition , we can verify if the complain is for service or a product.In NLP, real-world objects like products, places, and people are called named entities, and extracting them from text is called named entity recognition (NER).








In [52]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556571,Mega,208,212
4,PER,0.590256,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


The pipeline detected all the entities and also assigned a category such as ORG (organization), LOC (location), or PER (person) to each of them.

**QUESTION ANSWERING**

> In question answering, we provide the model with a passage of text called the context, along with a question whose answer we’d like to extract.



In [53]:
reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


 The pipeline returns the start and end integers that correspond to the character indices where the answer span was found along with the answer

 **SUMMARIZATION**

> The goal of text summarization is to take a long text as input and generate a short version with all the relevant facts.



In [54]:
summarizer = pipeline("summarization")
outputs = summarizer(text,  max_length=60, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead. As a lifelong enemy of the Decepticons, I hope you can understand


Model was able to capture the essence of the problem in the summary created

**TRANSLATION**

> Translation pipeline to translate text from a language to another
Below example transaltes text from English to German


In [55]:
translator = pipeline("translation_en_to_de",
                      model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt. Eingeschlossen sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, von Ihnen bald zu hören. Aufrichtig, Bumblebee.


**TEXT GENERATION**


> It can be used to provide fast replies like an autocomplete function



In [59]:
generator = pipeline("text-generation")

response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)

print(outputs[0]['generated_text'])


No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Dear Amazon, last week I ordered an Optimus Prime action figure
from your online store in Germany. Unfortunately, when I opened the package,
I discovered to my horror that I had been sent an action figure of Megatron
instead! As a lifelong enemy of the Decepticons, I hope you can understand my
dilemma. To resolve the issue, I demand an exchange of Megatron for the
Optimus Prime figure I ordered. Enclosed are copies of my records concerning
this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. However, this is something you should be able to do here at your own risk.

You may be able to return your issue after shipping, but you will get no return value. It is up to you, the customer, and the seller to determine if they cannot obtain the products
